In [1]:
import pandas as pd
import numpy as np
from gliner import GLiNER
import os
import sys 
cwd = os.getcwd()

# Add the '../scripts' directory to the system path
sys.path.insert(0, os.path.abspath(os.path.join(cwd, '../scripts')))
sys.path.insert(0, os.path.abspath(os.path.join(cwd, '../src')))
sys.path.insert(0, os.path.abspath(os.path.join(cwd, '../src/data_chunker')))

# Initialize GLiNER with the base model
model = GLiNER.from_pretrained("urchade/gliner_small-v2.1")


/Users/s.konchakova/Thesis/huber/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 4 files: 100%|██████████| 4/4 [03:10<00:00, 47.55s/it]
/Users/s.konchakova/Thesis/huber/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [75]:
df_subset = pd.read_csv('../../assets/csv/data_subset.csv', index_col=0)
df_subset.head()
#
text = df_subset['text'].iloc[4]
text

'The formal basis of doctoral studies is constituted by theDoctoral degree regulations of the School of Business and Economics (May 21, 2010). To register for doctoral studies at the School, please follow the following steps: Step 1: Supervision agreement If you are pursuing a structured doctoral degree programme, you must conclude a written supervision agreement once you have been successfully admitted to the programme by the relevant office. If you are pursuing an individual doctoral degree programme, you must conclude the supervision agreement with your supervisor directly. Step 2: Applying for admission to doctoral studies After concluding your supervision agreement, you must submit an application for admission to doctoral studies to the Graduate Office of the School of Business and Economics. The following documents are required: application for admission to doctoral studies copy of the supervision agreement an up-to-date CV copies of all university degrees (bachelor\'s and master

In [76]:
#check text column and substitute all ' [at] ' with '@'
df_subset['text'] = df_subset['text'].str.replace('(at)', '@')
text = df_subset['text'].iloc[134]
text

"Postal Address:Institute of Accounting and AuditingSchool of Business and EconomicsHumboldt-Universität zu BerlinUnter den Linden 6, 10099 Berlin Office: Dorotheenstraße 1, 10117 Berlin[How to find us] Our professors:Prof. Dr. Hanne BöckemProf. Dr. Ulf BrüggemannProf. Dr. Joachim Gassen Phone: +49 30 2093-99440 Fax: +49 30 2093-99441 E-Mail: wpruefung@wiwi.hu-berlin.de The TRR 266 Accounting for Transparency is a trans-regional Collaborative Research Center funded by the German Research Foundation (Deutsche Forschungsgemeinschaft – DFG). Our team of more than 100 dedicated researchers examines how accounting and taxation affect firm and regulatory transparency and how regulation and transparency impact our economy and society. We intend to help develop effective regulation for firm transparency and a transparent tax system. Naturally, we also ensure transparency of our own research. Aktuelles Latest News: 01.07.2024 - Seminare im Wintersemester 2024/25: Bewerbungsfristen / Seminars fo

In [77]:
# Define labels to look for
labels = ["person", "course", "date", "research_paper", "research_project", "teams", "city", "address", "organisation", "phone_number", "url", "other"]

# Predict entities in the text using the model
entities = model.predict_entities(text, labels)

# Print the number of entities found
print(f"Number of entities before deduplication: {len(entities)}")

# Remove duplicates by considering both 'text' and 'label'
unique_entities = { (entity['text'], entity['label']): entity for entity in entities }.values()

# Convert back to a list of dictionaries
unique_entities = list(unique_entities)

# Print the number of entities after deduplication
print(f"Number of unique entities after deduplication: {len(unique_entities)}")

# Optional: If you want to print out the unique entities
# for entity in unique_entities:
#     print(f"{entity['text']} => {entity['label']}")

unique_entities

/Users/s.konchakova/Thesis/huber/lib/python3.11/site-packages/gliner/data_processing/processor.py:269: UserWarning: Sentence of length 725 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")


Number of entities before deduplication: 15
Number of unique entities after deduplication: 14


[{'start': 107,
  'end': 118,
  'text': 'BerlinUnter',
  'label': 'city',
  'score': 0.7392323017120361},
 {'start': 179,
  'end': 185,
  'text': 'Berlin',
  'label': 'city',
  'score': 0.9862346053123474},
 {'start': 154,
  'end': 171,
  'text': 'Dorotheenstraße 1',
  'label': 'address',
  'score': 0.8000809550285339},
 {'start': 273,
  'end': 287,
  'text': 'Joachim Gassen',
  'label': 'person',
  'score': 0.5068864822387695},
 {'start': 295,
  'end': 312,
  'text': '+49 30 2093-99440',
  'label': 'phone_number',
  'score': 0.6214548349380493},
 {'start': 318,
  'end': 335,
  'text': '+49 30 2093-99441',
  'label': 'phone_number',
  'score': 0.5376328825950623},
 {'start': 476,
  'end': 502,
  'text': 'German Research Foundation',
  'label': 'organisation',
  'score': 0.7187440395355225},
 {'start': 504,
  'end': 535,
  'text': 'Deutsche Forschungsgemeinschaft',
  'label': 'organisation',
  'score': 0.5554184913635254},
 {'start': 538,
  'end': 541,
  'text': 'DFG',
  'label': 'organ

In [78]:
unique_entities = { (entity['text'], entity['label']): entity for entity in entities }.values()
unique_entities = list(unique_entities)


from collections import defaultdict

# Create a defaultdict to handle multiple entities with the same label
label_name_dict = defaultdict(list)

for entity in unique_entities:
    label_name_dict[entity['label']].append(entity['text'])

# Convert defaultdict to a regular dict if necessary
label_name_dict = dict(label_name_dict)

# Print the result to verify
label_name_dict

{'city': ['BerlinUnter', 'Berlin'],
 'address': ['Dorotheenstraße 1'],
 'person': ['Joachim Gassen'],
 'phone_number': ['+49 30 2093-99440', '+49 30 2093-99441'],
 'organisation': ['German Research Foundation',
  'Deutsche Forschungsgemeinschaft',
  'DFG',
  'Institut für Rechnungswesen und Wirtschaftsprüfung',
  'Institute of Accounting and Auditing'],
 'course': ['Bachelorseminar (Deutsch)',
  'Accounting (Englisch)',
  'Bachelorseminar']}

In [ ]:

from semantic_chunker import *
process_data_semantic(df_subset, model, labels)